In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
from shapely.geometry import Point, Polygon
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import descartes
import math
import matplotlib.colors as mcolors
import pyproj
import folium

In [2]:
wdir = "../Dades/AMS_Observacions/"
crs_latlon = 'EPSG:4326'  # WGS84
crs_utm = "EPSG:32631"

In [3]:
df_no2 = pd.read_csv('../Dades/AMS_Observacions/gene_sconcno2_2023_xvpca_emep_port.csv') 
df_estacions = pd.read_csv('../Dades/AMS_Observacions/XVPCA_info_sconcno2_2023.csv')


# Convert 'Date' column to datetime format
df_no2['Date'] = pd.to_datetime(df_no2['Date'])

# Pivot the dataframe to have each station ID as a column and the Date as the index
# columnas_es = [col for col in df_no2.columns if col.startswith('ES')]
columnas_es =df_no2.drop(['Date'], axis=1)


# Transpose the dataframe to have each station ID as a row and the Date as the column
df_no2_transposed = df_no2.set_index('Date').transpose().reset_index()

# Melt the dataframe to have a 'Value' column for each hour
df_no2_melted = pd.melt(df_no2_transposed, id_vars=['index'], var_name='Date', value_name='no2')

# Rename the 'index' column to 'Station'
df_no2_melted.rename(columns={'index': 'Station'}, inplace=True)

# Drop rows with NaN values
df_no2_melted.dropna(subset=['no2'], inplace=True)

print(df_no2_melted)

# Join the melted dataframe with the station information dataframe
df_obs = pd.merge(df_no2_melted, df_estacions, left_on='Station', right_on='code', how='inner')

# Drop the 'code' column as it is redundant
df_obs.drop(columns=['code','type'], inplace=True)
df_obs['geometry'] = gpd.points_from_xy(df_obs['lon'], df_obs['lat'], crs=crs_utm)
print(df_obs)

        Station                 Date   no2
2       ES1480A  2022-12-31 23:00:00  62.0
3       ES1396A  2022-12-31 23:00:00  60.0
4       ES1992A  2022-12-31 23:00:00  45.0
5       ES0691A  2022-12-31 23:00:00  43.0
6       ES0692A  2022-12-31 23:00:00  45.0
...         ...                  ...   ...
560633  ES2017A  2023-12-31 22:00:00   7.0
560634  ES1930A  2023-12-31 22:00:00   1.0
560635  ES1948A  2023-12-31 22:00:00   3.0
560636  ES1855A  2023-12-31 22:00:00   2.0
560637  ES1854A  2023-12-31 22:00:00   1.0

[529219 rows x 3 columns]
        Station                 Date   no2        lat       lon  \
0       ES1480A  2022-12-31 23:00:00  62.0  41.398762  2.153472   
1       ES1396A  2022-12-31 23:00:00  60.0  41.378803  2.133098   
2       ES1992A  2022-12-31 23:00:00  45.0  41.387273  2.115661   
3       ES0691A  2022-12-31 23:00:00  43.0  41.403716  2.204736   
4       ES0692A  2022-12-31 23:00:00  45.0  41.370760  2.114771   
...         ...                  ...   ...        ...  

In [4]:
# Cargar el shapefile
archivo_shapefile = "../Dades/OpenTransportMap/Barcelona/roadlinks_ES511.shp"
gdf_barcelona = gpd.read_file(archivo_shapefile)
gdf_girona = gpd.read_file("../Dades/OpenTransportMap/Girona/roadlinks_ES512.shp")
gdf_lleida = gpd.read_file("../Dades/OpenTransportMap/Lleida/roadlinks_ES513.shp")
gdf_tarragona = gpd.read_file("../Dades/OpenTransportMap/Tarragona/roadlinks_ES514.shp")

# Concatenate the GeoDataFrames
#gdf = pd.concat([gdf_barcelona, gdf_girona, gdf_lleida, gdf_tarragona], ignore_index=True)

gdf = gdf_barcelona


# Mostrar las primeras filas de la tabla de atributos
#print(gdf['functional'].head())

# Obtener los tipos únicos de la columna 'functional'
tipos_functional = gdf['functional'].unique()

# Asignar valores de prioridad a las categorías
valores_prioridad = {
    'mainRoad': 6 / 6,
    'firstClass': 5 / 6,
    'secondClass': 4 / 6,
    'thirdClass': 3 / 6,
    'fourthClass': 2 / 6,
    'fifthClass': 1 / 6,    
}

# Crear una nueva columna en el GeoDataFrame con los valores de prioridad
gdf['prioridad'] = gdf['functional'].map(valores_prioridad)

# Crear un nuevo DataFrame con las columnas 'sourceid', 'functional' y 'prioridad'
df_transport = gdf[['sourceid', 'prioridad','geometry']].drop_duplicates().reset_index(drop=True)
# Simplificar la geometría
df_transport['geometry'] = df_transport['geometry'].simplify(tolerance=0.01, preserve_topology=True)

print(df_transport.head())

# Guardar el GeoDataFrame en un archivo CSV
#gdf.to_csv("./carreteres.csv", index=False)


    sourceid  prioridad                                         geometry
0  198794320   0.166667  LINESTRING (1.99488 41.66973, 1.99502 41.66971)
1  198935562   0.333333  LINESTRING (2.23387 41.94304, 2.23386 41.94206)
2  198944072   0.333333  LINESTRING (1.68342 41.33341, 1.68353 41.33318)
3  198965364   0.166667  LINESTRING (2.01923 41.57089, 2.01926 41.57088)
4  198965365   0.166667  LINESTRING (2.01857 41.57034, 2.01852 41.57033)


In [ ]:
from multiprocessing import Pool


stations = gpd.GeoDataFrame(df_obs[['Station', 'geometry']].drop_duplicates().reset_index(drop=True), geometry='geometry')
stations['buffer'] = stations.geometry.buffer(100)

# Leer las carreteras desde el DataFrame
carreteras = df_transport

# Crear un índice espacial para las carreteras
carreteras_sindex = carreteras.sindex

def calcular_interseccion(buffer):
    # Encontrar las carreteras que intersectan con el buffer
    posibles_intersecciones = list(carreteras_sindex.intersection(buffer.bounds))
    carreteras_intersectadas = carreteras.iloc[posibles_intersecciones]
    
    # Calcular la intersección
    intersecciones = carreteras_intersectadas[carreteras_intersectadas.intersects(buffer)]
    
    if not intersecciones.empty:
        intersecciones['longitud_interseccion'] = intersecciones.intersection(buffer).length
        intersecciones['porcentaje'] = (intersecciones['longitud_interseccion'] / intersecciones.length) * 100
        intersecciones['buffer_id'] = buffer.name
        
        return intersecciones[['buffer_id', 'sourceid', 'porcentaje']]
    else:
        return pd.DataFrame(columns=['buffer_id', 'sourceid', 'porcentaje'])

# Procesar los buffers en paralelo
with Pool() as pool:
    resultados = pool.map(calcular_interseccion, stations['buffer'])

# Combinar los resultados
resultados_df = pd.concat(resultados, ignore_index=True)

# Mostrar resultados
print(resultados_df)